# SQL
## Contexto: 
Você tem a tabela “vendas” com as colunas:
- id_venda (int);
- id_cliente (int); 
- data_venda (date); 
- valor (decimal);
- canal (varchar: 'online', 'loja', 'parceiro').


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Criar dados sintéticos em Pandas
np.random.seed(42)
n = 1000  # número de vendas sintéticas

# Gerar colunas
id_venda = np.arange(1, n+1)
id_cliente = np.random.randint(101, 201, size=n)  # 100 clientes distintos
data_venda = pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2025-09-01"), size=n))
valor = np.round(np.random.uniform(10, 1000, size=n), 2)
canal = np.random.choice(['online', 'loja', 'parceiro'], size=n, p=[0.5, 0.3, 0.2])

# Criar DataFrame
vendas_df = pd.DataFrame({
    'id_venda': id_venda,
    'id_cliente': id_cliente,
    'data_venda': data_venda,
    'valor': valor,
    'canal': canal
})

# Converter para Spark DataFrame
vendas_df = spark.createDataFrame(vendas_df)
vendas_df.createOrReplaceTempView("vendas")
display(vendas_df)


In [0]:
np.random.seed(42)
n_clientes = 120  # 100 clientes com vendas + 20 extras

id_clientes = np.arange(101, n_clientes+101)

# Listas simples de nomes e sobrenomes
primeiros_nomes = ["Ana", "Bruno", "Carlos", "Daniela", "Eduardo", "Fernanda", "Gabriel", "Helena", "Igor", "Julia"]
sobrenomes = ["Silva", "Santos", "Oliveira", "Souza", "Costa", "Pereira", "Rodrigues", "Almeida", "Lima", "Gomes"]

# Gerar nomes aleatórios combinando primeiro nome + sobrenome
nomes = [f"{np.random.choice(primeiros_nomes)} {np.random.choice(sobrenomes)}" for _ in range(n_clientes)]

# Criar pandas dataframe
df_clientes = pd.DataFrame({
    'id_cliente': id_clientes,
    'nome': nomes
})

# Criar Spark DataFrame
df_clientes = spark.createDataFrame(df_clientes)
df_clientes.createOrReplaceTempView("clientes") # Vista temporária (SQL)

# Mostrar os dados
display(df_clientes)

## Questões: 

1. Escreva uma query em SQL que retorne o valor total vendido por canal em 2024. 
2. Escreva uma query em SQL que traga os 3 clientes que mais compraram (soma de valor) no ano de 2024. 
3. Escreva uma query em SQL que calcula o ticket médio (média de valor por cliente) no canal 'online' em janeiro de 2025. 
4. Escreva uma query em SQL que liste todos os clientes que não compraram nada em 2024, assumindo que existe uma tabela “clientes” com as colunas id_cliente e nome.

In [0]:
%sql
SELECT 
    canal,
    ROUND(SUM(valor), 3) AS total_vendido
FROM vendas
WHERE YEAR(data_venda) = 2024
GROUP BY canal
ORDER BY total_vendido DESC;

In [0]:
%sql
SELECT 
    id_cliente,
    ROUND(SUM(valor), 3) AS total_comprado
FROM vendas
WHERE YEAR(data_venda) = 2024
GROUP BY id_cliente
ORDER BY total_comprado DESC
LIMIT 3;

In [0]:
%sql
SELECT 
    ROUND(AVG(valor), 3) AS avg_ticket
FROM vendas
WHERE canal = 'online'
  AND YEAR(data_venda) = 2025
  AND MONTH(data_venda) = 1;

In [0]:
%sql
SELECT 
    c.id_cliente,
    c.nome
FROM clientes c
LEFT JOIN vendas v
    ON c.id_cliente = v.id_cliente
   AND YEAR(v.data_venda) = 2024
WHERE v.id_cliente IS NULL;